In [7]:
import os
import datetime
import time
import shutil
import numpy as np
import requests
import pprint
import queue
import urllib.request
from html.parser import HTMLParser
from urllib.parse import urlparse
import urllib.robotparser
import heapq

In [33]:
def url_score(query, url):
    query_ws = query.lower().split()
    # because geometry mean will be used
    url_score = 0
    for w in query_ws:
        if len(w) > 3:
            for j in range(3, len(w) + 1):
                if w[:j] in url.lower():
                    url_score += 1
        else:
            if w in url.lower():
                    url_score += 1
    if url_score > len(query_ws):
        return url_score
    else:
        return 0

In [43]:
num = 0
with open('wildfires_california_bfs.txt', 'r') as f:
    lines = f.read().split('\n')
    for l in lines[:1000]:
        score = l.split(',')[-1]
        score = float(score)
        if score >= 1:
            num += 1
num

219

In [34]:
url_score('wildfires california', 'https://pinterest.com/theoregonian')

0

In [37]:
time.sleep(10)
t2 = datetime.datetime.now()
print(t2 - t1)

0:00:10.000439


In [28]:
shutil.rmtree('focus_crawled')

In [38]:
str(datetime.datetime.now())

'2018-09-25 15:47:35.840048'

In [2]:
query = 'dog'

In [3]:
url = 'https://www.gigablast.com/search'
params = {
    'format': 'json',
    'q': query,
    'c': 'main',
    'n': 10,
    'showerrors': 1,
    'userid':196,
    'code':1489906050,
}
res = requests.get(url, params = params)

In [4]:
results = res.json()['results']
pprint.pprint(results)

[{'contentHash32': 1306546529,
  'contentType': 'html',
  'docId': 90709260389,
  'docScore': 23741399040.0,
  'firstIndexedDateUTC': 1513141581,
  'fromHostId': 37,
  'fromShard': 37,
  'hopCount': 2,
  'isAdult': 0,
  'langAbbr': 'en',
  'language': 'English',
  'site': 'www1.maine.gov',
  'size': '   0k',
  'sizeInBytes': 18,
  'spidered': 1513141581,
  'sum': 'ERR_CONNECT_FAIL.',
  'summaryScoreTimeMS': 0,
  'summaryTagdbLookupTimeMS': 0,
  'summaryTitleRecLoadTimeMS': 1,
  'title': 'dog license online',
  'totalSummaryGetTimeMS': 3,
  'url': 'http://www1.maine.gov/cgi-bin/online/dog_license/index.pl'},
 {'contentHash32': 0,
  'contentType': 'html',
  'docId': 212337244226,
  'docScore': 23741399040.0,
  'firstIndexedDateUTC': 1511872668,
  'fromHostId': 28,
  'fromShard': 28,
  'hopCount': 0,
  'isAdult': 0,
  'langAbbr': 'xx',
  'language': 'Unknown',
  'site': 'www1.maine.gov',
  'size': '   0k',
  'sizeInBytes': 0,
  'spidered': 1521716652,
  'sum': '',
  'summaryScoreTimeMS': 

In [5]:
class MyHTMLParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.links = []
        self.lines = []
        self.base = ''
        
    def handle_starttag(self, tag, attrs):
        for name, value in attrs:
            if name == 'href' and not value.split('.')[-1] in ['css', 'jpg', 'png', 'ico', 'mp3', 'mp4']:
                if tag == 'base':
                    self.base = value
                    continue
                o = urlparse(value)
                # TODO: relative url
                if o.scheme in ['http', 'https']:
                    self.links.append(value)
                
    def handle_endtag(self, tag):
        pass

    def handle_data(self, data):
        if data.strip() and '{' not in data:
            self.lines.append(data.strip())

In [6]:
def parse_page(filename):
    with open(filename, 'r', encoding = 'UTF-8', errors = 'ignore') as f:
        size = os.path.getsize(filename)
        parser = MyHTMLParser()
        parser.feed(f.read())
    return parser.lines, list(set(parser.links)), size

In [7]:
def can_visit(url):
    o = urlparse(url)
    rb_url = o.scheme + '://' + o.netloc + '/robots.txt'
    rp = urllib.robotparser.RobotFileParser()
    rp.set_url(rb_url)
    try:
        # might denied
        rp.read()
        return rp.can_fetch('*', url)
    except:
        return False

In [8]:
def occurence(word, lines):
    num = 0
    for line in lines:
        for w in line.split():
            if word.lower() == w.lower():
                num += 1
    return num

def score(query, url, lines):
    query_ws = query.lower().split()
    occrs = {}
    for line in lines:
        for w in line.split():
            if w.lower() in query_ws:
                occrs[w.lower()] = occrs.get(w.lower(), 1) + 1
    url_score = 1
    for w in query_ws:
        if w in url:
            url_score += 1
    # use geometry mean
    return np.sqrt(url_score * sum(occrs.values()))

In [9]:
def normalize_url(url):
    l = url.split('//')
    return ''.join(l[1:])

In [10]:
def check_header(headers):
    for k in headers.keys():
        if k.lower() == 'content-type' and 'text/html' in headers[k]:
            return True
    return False

In [40]:
normalize_url('https://www.youtube.com/embed/IOYyCHGWJq4')

'www.youtube.com/embed/IOYyCHGWJq4'

In [11]:
MAX_NUM = 100
MAX_PER_PAGE = 3

In [3]:
os.mkdir('ff')

In [24]:
class Crawler():
    def __init__(self):
        self.log = []
        self.parsed_num = 0
        self.num_404 = 0
        
    def reset(self):
        self.log = []
        self.parsed_num = 0
        self.num_404 = 0
        
        
    def bfs(self, query, MAX_NUM, MAX_PER_PAGE):
        self.reset()
        if os.path.exists('bfs_crawled'):
            shutil.rmtree('bfs_crawled')
        os.mkdir('bfs_crawled')
        Q = queue.Queue()
        visited = {}
        for result in results:
            Q.put(
                {
                    'url': result['url'],
                    'depth': 0,
                }
            )
            url = normalize_url(result['url'])
            visited[url] = 0

        list_result = []
        while self.parsed_num < MAX_NUM:
            # robot exclusion
            cur = Q.get()
            print('*' * 10)
            cur_url = cur['url']
            print(cur_url)
            if not can_visit(cur_url):
                continue
            # retrieve file
            try:
                local_filename, headers = urllib.request.urlretrieve(cur_url, filename = 'bfs_crawled/file' + str(self.parsed_num) +'.txt')
            # handle Forbidden error
            except urllib.error.HTTPError as err:
                print(err)
                if '404' in str(err):
                    self.num_404 += 1
            else:
                if not check_header(headers):
                    continue
                print('downloaded')
                print(self.parsed_num)
                time = str(datetime.datetime.now())
                print('time', time)
                lines, urls, size = parse_page(local_filename)
                print('size', size)
                # calculate the relevance
        #         cur_score = occurence(query, lines)
                cur_score = score(query, cur_url, lines)
                print('relevance', cur_score)
                # record the page
                self.parsed_num += 1
                self.log.append(
                    (
                        cur_url, time, size, '200', 'bfs', cur_score,
                    )
                )
                # BFS
                i = 0
                for url in urls[:min(len(urls), MAX_PER_PAGE)]:
                    print(i)
                    i += 1
                    if normalize_url(url) in visited:
                        # plain
                        continue
                    Q.put(
                        {
                            'url': url,
                            'depth': cur['depth'] + 1,
                        }
                    )
                    visited[normalize_url(url)] = cur['depth'] + 1
                    
    
    def focus(self, query, MAX_NUM, MAX_PER_PAGE):
        self.reset()
        if os.path.exists('focus_crawled'):
            shutil.rmtree('focus_crawled')
        os.mkdir('focus_crawled')
        # initial the queue
        heap = []
        visited = {}
        for result in results:
            heapq.heappush(heap, (0, result['url']))
            # normalize url
            url = normalize_url(result['url'])
            visited[url] = 0
            
        level = 0
        while self.parsed_num < MAX_NUM:
            d_childs = {}
            d_scores = {}
            while len(heap):
                prior_score, cur_url = heapq.heappop(heap)
                print('*' * 10)
                print(cur_url)
                # robot exclusion
                if not can_visit(cur_url):
                    continue
                # retrieve file
                try:
                    local_filename, headers = urllib.request.urlretrieve(cur_url, filename = 'focus_crawled/file' + str(self.parsed_num) + '.txt')
                # handle Forbidden error
                except urllib.error.HTTPError as err:
                    print(err)
                    if '404' in str(err):
                        self.num_404 += 1
                else:
                    if not check_header(headers):
                        continue
                    print('downloaded')
                    time = str(datetime.datetime.now())
                    print('time', time)
                    lines, urls, size = parse_page(local_filename)
                    print('size', size)
                    prior_score = -prior_score
                    print('estimate', prior_score)
                    # record the page
                    self.parsed_num += 1
                    # calculate the relevance
        #             cur_score = occurence(query, lines)
                    cur_score = score(query, cur_url, lines)
                    print('relevance', cur_score)
                    self.log.append(
                        (
                            cur_url, time, size, '200', prior_score, cur_score,
                        )
                    )
                    d_childs[cur_url] = []
                    for url in urls:
                        if url in visited:
                            continue
                        elif url in d_scores:
                            # update the estimated promise
                            d_scores[url].append(cur_score)
                            continue
                        else:
                            d_scores[url] = [cur_score]
                        d_childs[cur_url].append(url)

            level += 1
            # calculate the promise
            for k in d_scores:
                v = d_scores[k]
                # use the average score
                d_scores[k] = sum(v) / len(v)
            for k in d_childs:
                v = d_childs[k]
                d_childs[k] = sorted(v, key = lambda x:d_scores[x])
            # push to queue
            for k, urls in d_childs.items():
                for url in urls[:min(len(urls), MAX_PER_PAGE)]:
                    heapq.heappush(heap, (-d_scores[url], url))
                    visited[normalize_url(url)] = level

# BFS baseline

In [13]:
crawler = Crawler()
crawler.bfs(query, MAX_NUM, MAX_PER_PAGE)

**********
http://www1.maine.gov/cgi-bin/online/dog_license/index.pl
**********
https://www1.maine.gov/cgi-bin/online/dog_license/index.pl
**********
https://www.reddit.com/r/funny/comments/7ex34w/optical_illusion_dog/
downloaded
0
time 2018-09-26 00:12:09.672635
size 378537
relevance 5.291502622129181
0
1
2
**********
https://twitter.com/hashtag/dog?src=hash
downloaded
1
time 2018-09-26 00:12:10.537385
size 217689
relevance 0.0
0
1
2
**********
https://www.redhouserecords.com/store/dog
downloaded
2
time 2018-09-26 00:12:11.651162
size 45278
relevance 3.7416573867739413
0
1
2
**********
https://dogvacay.com/
downloaded
3
time 2018-09-26 00:12:13.590911
size 51705
relevance 4.47213595499958
0
1
2
**********
https://www.terrificpets.com/dog_breeds/
**********
http://www.dogshaming.com/
downloaded
4
time 2018-09-26 00:12:14.499142
size 49251
relevance 2.449489742783178
0
1
2
**********
http://knowyourmeme.com/memes/this-is-fine
downloaded
5
time 2018-09-26 00:12:17.482972
size 110777
rele

downloaded
47
time 2018-09-26 00:13:19.005736
size 62754
relevance 7.280109889280518
0
1
2
**********
https://www.naturesrecipe.com/dog-recipes/grain-free/chicken-and-venison-recipe-in-broth
downloaded
48
time 2018-09-26 00:13:20.190612
size 76293
relevance 4.69041575982343
0
1
2
**********
https://twitter.com/roverdotcom?lang=bn
downloaded
49
time 2018-09-26 00:13:21.228500
size 447378
relevance 3.4641016151377544
0
1
2
**********
https://twitter.com/roverdotcom?lang=ur
downloaded
50
time 2018-09-26 00:13:22.604195
size 432629
relevance 3.4641016151377544
0
1
2
**********
https://twitter.com/roverdotcom?lang=bg
downloaded
51
time 2018-09-26 00:13:24.104400
size 434276
relevance 3.4641016151377544
0
1
2
**********
http://blog.soundcloud.com
**********
http://soundcloud.tumblr.com/tagged/soundcloud
downloaded
52
time 2018-09-26 00:13:24.894319
size 45728
relevance 0.0
0
1
2
**********
http://soundcloud.tumblr.com/tagged/jan-blomqvist
downloaded
53
time 2018-09-26 00:13:25.495215
size 28

downloaded
95
time 2018-09-26 00:14:38.884888
size 260971
relevance 0.0
0
1
2
**********
https://www.consumer.ftc.gov/health
downloaded
96
time 2018-09-26 00:14:39.222651
size 28171
relevance 0.0
0
1
2
**********
https://www.consumer.ftc.gov/articles/0025-kids-and-mobile-phones
downloaded
97
time 2018-09-26 00:14:39.535122
size 41152
relevance 0.0
0
1
2
**********
http://www.ftc.gov/
downloaded
98
time 2018-09-26 00:14:40.147412
size 108234
relevance 0.0
0
1
2
**********
https://europa.eu/european-union/index_en
downloaded
99
time 2018-09-26 00:14:42.173798
size 117483
relevance 0.0
0
1
2


In [16]:
sum([l[-1] for l in crawler.log])

141.06648933333406

# Focus Crawler

In [25]:
crawler = Crawler()
crawler.focus(query, MAX_NUM, MAX_PER_PAGE)

**********
http://knowyourmeme.com/memes/this-is-fine
downloaded
time 2018-09-26 00:24:15.044393
size 110777
estimate 0
relevance 3.1622776601683795
**********
http://www.channel4.com/programmes/rich-dog-poor-dog/on-demand/59951-001
downloaded
time 2018-09-26 00:24:15.643639
size 19223
estimate 0
relevance 2.449489742783178
**********
http://www.dogshaming.com/
downloaded
time 2018-09-26 00:24:16.216693
size 50640
estimate 0
relevance 2.449489742783178
**********
http://www1.maine.gov/cgi-bin/online/dog_license/index.pl
**********
https://dogvacay.com/
downloaded
time 2018-09-26 00:24:38.837272
size 51705
estimate 0
relevance 4.47213595499958
**********
https://twitter.com/hashtag/dog?src=hash
downloaded
time 2018-09-26 00:24:39.699060
size 217689
estimate 0
relevance 0.0
**********
https://www.reddit.com/r/funny/comments/7ex34w/optical_illusion_dog/
downloaded
time 2018-09-26 00:24:41.249722
size 368517
estimate 0
relevance 5.291502622129181
**********
https://www.redhouserecords.com/

downloaded
time 2018-09-26 00:25:48.559002
size 45251
estimate 6.355576334814152
relevance 3.605551275463989
**********
https://support.rover.com/hc/en-us
downloaded
time 2018-09-26 00:25:49.324038
size 135329
estimate 5.656854249492381
relevance 5.656854249492381
**********
https://www.rover.com/community/questions/scope:unanswered/sort:activity-desc/tags:support/page:1/
downloaded
time 2018-09-26 00:25:50.473663
size 206061
estimate 5.656854249492381
relevance 2.8284271247461903
**********
https://soundcloud.com/red-house-records
downloaded
time 2018-09-26 00:25:51.648755
size 26493
estimate 3.7416573867739413
relevance 0.0
**********
https://www.redhouserecords.com/sites/default/files/tracklists/%5Bnode%3Atitle%5D/USRHR1730009.mp3?uuid=5baace3079c46
**********
https://www.redhouserecords.com/store/dog
downloaded
time 2018-09-26 00:25:59.458008
size 45278
estimate 3.7416573867739413
relevance 3.7416573867739413
**********
http://www.rover.com/contact/
downloaded
time 2018-09-26 00:26

**********
https://www.dogshaming.com/wp-json/
**********
http://www.youtube.com/watch?v=ixZqFxeET4k
downloaded
time 2018-09-26 00:27:09.474277
size 196016
estimate 1.8708286933869707
relevance 1.4142135623730951
**********
https://apps.mzstatic.com/content/static-config/android/manifest.json
**********
https://itunes.apple.com/us/app/lime-your-ride-anytime/id1199780189?mt=8
downloaded
time 2018-09-26 00:27:11.780535
size 226400
estimate 1.8027756377319946
relevance 0.0
**********
https://itunes.apple.com/us/app/turo-better-than-car-rental/id555063314?mt=8
downloaded
time 2018-09-26 00:27:12.879008
size 219549
estimate 1.8027756377319946
relevance 0.0
**********
https://m.youtube.com/watch?v=ixZqFxeET4k
downloaded
time 2018-09-26 00:27:13.976560
size 196290
estimate 1.4142135623730951
relevance 0.0
**********
https://www.youtube.com/opensearch?locale=en_US
**********
https://www.youtube.com/watch?v=ixZqFxeET4k
downloaded
time 2018-09-26 00:27:15.152280
size 196889
estimate 0.7071067811

In [27]:
sum([l[-1] for l in crawler.log])

282.35259342851845